In [23]:
import torch


class TestModel(torch.nn.Module):
    
    def __init__(self):
        super(TestModel, self).__init__()
        
        self.fc1 = torch.nn.Linear(2, 2)
        self.fc2 = torch.nn.Linear(2, 1)
        
        self.relu = torch.nn.ReLU()
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x

loss = torch.nn.MSELoss()

test_input = torch.tensor([[1.0, 1.0], [0.0, 0.0], [1.0, 0.0], [0.0, 1.0]])
test_label = torch.tensor([[0.0], [0.0], [1.0], [1.0]])

first_model = TestModel()
second_model = TestModel()

first_loss = loss(first_model(test_input), test_label)
second_loss = loss(second_model(test_input), test_label)

print("First model loss: ", first_loss.item())
print("Second model loss: ", second_loss.item())



First model loss:  0.4339907467365265
Second model loss:  0.40908360481262207


In [24]:
class ReconstructionDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    

## Random Gaussian Noise for Dataset
data = torch.normal(0, 1, (50, 2))
## 1 or 0 as labels
labels = torch.randint(0, 2, (50, 1)).float()
                      
original_dataset = ReconstructionDataset(data, labels)
original_dataloader = torch.utils.data.DataLoader(original_dataset, batch_size=10, shuffle=True)

In [32]:
import copy
training_rate = 0.01 * 60
reconstruction_iter = 10

def original_reconstruct_function(model_0, model_1) :
    
    model_0_params = dict(model_0.named_parameters())
    model_1_params = dict(model_1.named_parameters())
    
    mean_gradient_per_param ={}
    for name, param1 in model_0_params.items():
        param2 = model_1_params[name]
        parameter_diff = (param2 - param1)
        mean_gradient = parameter_diff / training_rate
        mean_gradient_per_param[name] = mean_gradient.detach()
        
    dataloader = copy.deepcopy(original_dataloader)
    criterion = torch.nn.MSELoss()
    
    for data, label in dataloader:
        
        generated_data = data.clone().detach()
        generated_data.requires_grad = True
        optimizer = torch.optim.SGD([generated_data], lr=0.01)
        def closure():
           
            optimizer.zero_grad()
            output = model_1(generated_data)
            loss = criterion(output, label)
            dummy_gradient = torch.autograd.grad(loss, model_1.parameters(), create_graph=True)
            
            l2_loss = 0
            for dummy_grad, mean_grad in zip(dummy_gradient, mean_gradient_per_param.values()):
                l2_loss += ((dummy_grad - mean_grad) ** 2).sum()             ## float32 - float32
            l2_loss.backward()

            return l2_loss
        
        for i in range(reconstruction_iter):
            optimizer.step(closure)
            #print(f"{i}th l2 loss : {closure()}")
            

def vertical_l2_loss_reconstruct_function(model_0, model_1) :
    
    model_0_params = dict(model_0.named_parameters())
    model_1_params = dict(model_1.named_parameters())
    
    mean_gradient_per_param ={}
    for name, param1 in model_0_params.items():
        param2 = model_1_params[name]
        parameter_diff = (param2 - param1)
        mean_gradient = parameter_diff / training_rate
        mean_gradient_per_param[name] = mean_gradient.detach()
    
    stacked_mean_gradient = torch.stack(list(mean_gradient_per_param.values()))
        
    dataloader = copy.deepcopy(original_dataloader)
    criterion = torch.nn.MSELoss()
    
    for data, label in dataloader:
        
        generated_data = data.clone().detach()
        generated_data.requires_grad = True
        optimizer = torch.optim.SGD([generated_data], lr=0.01)
        def closure():
           
            optimizer.zero_grad()
            output = model_1(generated_data)
            loss = criterion(output, label)
            dummy_gradient = torch.autograd.grad(loss, model_1.parameters(), create_graph=True)
            stacked_dummpy_gradient = torch.stack(dummy_gradient)
            
            l2_loss = ((stacked_dummpy_gradient - stacked_mean_gradient) ** 2).sum()
            
            l2_loss.backward()

            return l2_loss
        
        for i in range(reconstruction_iter):
            optimizer.step(closure)
            #print(f"{i}th l2 loss : {closure()}")
    
    

In [33]:
## time the reconstruction of the input
import time

reconstruction_start_time = time.time()

original_reconstruct_function(first_model, second_model)
reconstruction_end_time = time.time()
original_time = reconstruction_end_time - reconstruction_start_time
#print(f"Reconstruction time: {reconstruction_end_time - reconstruction_start_time}")

reconstruction_start_time = time.time()
vertical_l2_loss_reconstruct_function(first_model, second_model)
reconstruction_end_time = time.time()
vertical_time = reconstruction_end_time - reconstruction_start_time

time_diff = original_time - vertical_time


RuntimeError: stack expects each tensor to be equal size, but got [2, 2] at entry 0 and [2] at entry 1

In [27]:
def reconstruct_sequence_embedding_via_label(self,
                                            labels : List[torch.Tensor],
                                            list_of_item_seq_len : List[torch.Tensor],
                                            current_model : torch.nn.Module,
                                            previous_model : torch.nn.Module,
                                            epoch : int,
                                            learning_rate : float,
                                            criterion_type : str,
                                            record_history : bool = False) -> Dict[torch.Tensor, Dict[str, any]]:
        """
        reconstruct gaussian noise to match the right x_embedding for label
        
        The original Deep Leakage easedrops on normal gradient, and infer the training data.
        In this algorithm we make assumptions that normal gradient is the difference between
        current model and previous model.
        """
        
        generated_data = {"input_embedding" : [],
                          "item_seq_len" : [],
                          "original_label" : [],
                          "target_item" : [],
                          "history" : []}
        training_rate = epoch * learning_rate
        
        previous_model_params = dict(previous_model.named_parameters())
        current_model_params = dict(current_model.named_parameters())
        
        criterion = torch.nn.CrossEntropyLoss()
        
        mean_gradient_per_param ={}
        for name, param1 in previous_model_params.items():
            param2 = current_model_params[name]
            parameter_diff = (param2 - param1)
            mean_gradient = parameter_diff / training_rate
            mean_gradient_per_param[name] = mean_gradient.detach()
        
        reconstruction_loader = self._get_reconstruction_loader(labels, self._cfg.attack.reconstruction_batch_size)
        
        ## per batch generate
        for batch in reconstruction_loader:
            label = batch['label']
            input_embedding = batch['input_embedding']
            item_seq_len = batch['item_seq_len']
            
            generated_input_embedding = input_embedding.clone().detach()
            generated_input_embedding.requires_grad_(True)
            
            optimizer = torch.optim.LBFGS([generated_input_embedding])
            
            history = {}
            ## initialize history section via labels
            for single_label in label:
                history[single_label.item()] = {"input_embedding" : [], "loss" : []}
                
            for iter in range(self._cfg.attack.reconstruction_iter):
                def closure():
                    
                    optimizer.zero_grad()
                    outputs = current_model.embedding_forward(generated_input_embedding, item_seq_len)
                    test_item_emb = current_model.item_embedding.weight
                    logits = torch.matmul(outputs, test_item_emb.transpose(0,1))
                    loss = criterion(logits, label)
                    dummy_gradient = torch.autograd.grad(loss, current_model.parameters(), create_graph=True) ## Tuple of tensors
                    
                    l2_loss = 0
                    ## Since this is mean_gradient_per_param is dict it value is the tensor.
                    for dummy_grad, mean_grad in zip(dummy_gradient, mean_gradient_per_param.values()):
                        l2_loss += ((dummy_grad - mean_grad) ** 2).sum()
                        ## float32 - float32
                    l2_loss.backward()
                    
                    return l2_loss
                    
                optimizer.step(closure)
                
                ## Try to Save every 10 iter
                if (iter + 1) % 10 == 0 and record_history:
                    current_grad_diff = closure()
                    for single_label in label:
                        history[single_label.item()]["input_embedding"].append(generated_input_embedding.clone().detach())
                        history[single_label.item()]["loss"].append(current_grad_diff.clone().detach())
                    
            ## decompose the batch into single instance and
            ## append to the generated_data
            for i in range(len(label)):
                generated_data["input_embedding"].append(generated_input_embedding[i].unsqueeze(0).clone().detach()) # List[torch.Tensor]
                generated_data["item_seq_len"].append(item_seq_len[i]) # List[torch.Tensor]
                generated_data["original_label"].append(label[i]) # List[torch.Tensor]
                generated_data["target_item"].append(torch.zeros_like(item_seq_len[i]) + self._cfg.attack.target_item_id) # List[torch.Tensor]
                generated_data["history"].append(history[label[i].item()]) # List[Dict[str, List[torch.Tensor]]]
       
        
        return generated_data

NameError: name 'List' is not defined